Prediction of the melt pool type using CNN
==============================================
**Author**: `Zaixi Shang, Yaxin Cui and Chengyang Wu`

In [1]:
%matplotlib inline

In [2]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from skimage import io, transform

import copy
import os.path
import glob
from torch.utils.data import Dataset, DataLoader
import pandas as pd
plt.ion()   # interactive mode
from matplotlib.pyplot import imshow


Load Data
---------
Load in pre-processed images and labels in training set and validation set. The pre-processed image contains the information of command line, and the lables are the melt pool types (zero, small, normal and large).

In [3]:
# Writing a custermized dataset for my data
class CommendDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, root_dir , label_dir  , phase, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
#         self.num_image = glob.glob(os.path.join(root_dir,'*.png'))
        if phase == 'train':
            self.num_image = 40000
        else:
            self.num_image = 10000
        self.df_label = pd.read_csv(os.path.join(label_dir, phase + '.csv'))
#         self.num_image=self.df_label.shape[0]
        self.phase = phase

    def __len__(self):
#         return len(self.num_image)
        return self.num_image

    def __getitem__(self, idx):
        lab = self.df_label.iloc[idx]['category']
        # if self.phase == 'val':
        #     idx = idx+1999
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                str(idx)+'.png')
        # image = io.imread(img_name)
        # image = image[20:40,20:40,:].transpose((2, 0, 1))
        
        image = io.imread(img_name).transpose((2, 0, 1))

        if self.transform:
            image = self.transform(image)
        
        return image,lab

In [4]:
# Data augmentation and normalization for training
data_transforms = transforms.Compose([
        transforms.CenterCrop(40)
    ])
data_dir = './layers/'
label_dir = './layers/'
image_datasets = {x: CommendDataset(root_dir = os.path.join(data_dir, x),label_dir =label_dir,transform=None ,phase = x) for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=200,
                                             shuffle=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Training the model
------------------


### Define the classification model

In [25]:
def train_model_class(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
#         print('Epoch {}/{}'.format(epoch, num_epochs - 1))
#         print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
                
            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data.
            epo_count = 0
            for inputs, labels in dataloaders[phase]:
                epo_count += 1
                inputs = inputs.to(device, dtype=torch.float32)
                labels = labels.to(device, dtype=torch.float32)
                
                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    labels = labels.long()
                    loss = criterion(outputs, labels)
                    
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)    
#                 epoch_loss += loss * inputs.size(0)
#                 if (epo_count % 10) == 0:
#                     num_sample = inputs.size(0)*epo_count
#                     cur_RMSE = epoch_loss/num_sample
#                     cur_RMSE = cur_RMSE.sqrt()
#                     print('{} number of samples: {} RMSE: {:.4f}'.format(phase, num_sample, cur_RMSE))
#                     if phase == 'val' and cur_RMSE < best_acc:
#                         best_acc = cur_RMSE
#                         best_model_wts = copy.deepcopy(model.state_dict()) 
#                         print('Best performace, saving model...')
#                         torch.save(best_model_wts,'./state/model.p')
#                 # statistics
#                 # running_loss += loss.item() * inputs.size(0)
#                 # running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()
                
            epoch_loss = running_loss/dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(best_model_wts,'./state/classifactionalexnet.p')
        print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

### Define the CNN model

In [7]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        # our dimension [3,40,40]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 20, 20]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 10, 10]
            nn.ZeroPad2d(1), # [64,12,12]
    

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 12, 12]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 6, 6]
            nn.ZeroPad2d(1), # [128,8,8]
    

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 8, 8]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 4, 4]
 

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 2, 2]
            
#             nn.Conv2d(512, 512, 3, 1, 1), # [512, 4, 4]
#             nn.BatchNorm2d(512),
#             nn.ReLU(),
#             nn.MaxPool2d(2, 2, 0),       # [512, 2, 2]
        )
        self.cnn = nn.DataParallel(self.cnn)
        self.fc = nn.Sequential(
            nn.Linear(512*2*2, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 4)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

Running the model
----------------------




In [8]:
num_ca = 4
model_use = 'resnext50_32x4d'

if model_use == 'resnet18': 
    model_ft = models.resnet18(pretrained=False)
    num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
    model_ft.fc = nn.Linear(num_ftrs,num_ca)
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

elif model_use == 'alexnet': 
    model_ft = models.alexnet(pretrained=True)
    num_ftrs = model_ft.classifier[-1].in_features
    model_ft.classifier[-1] = nn.Linear(num_ftrs,num_ca)
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
    
elif model_use == 'diy':
    model_ft = Classifier()
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=1e-5, momentum=0.9)
    
elif model_use == 'resnext50_32x4d': 
    model_ft = models.resnext50_32x4d(pretrained=False)
    num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
    model_ft.fc = nn.Linear(num_ftrs,num_ca)
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

model_ft = model_ft.to(device)


criterion = nn.CrossEntropyLoss()
 

# Observe that all parameters are being optimized

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=4, gamma=0.1)

### Running Model

In [35]:
model_ft = train_model_class(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                             num_epochs=5)

tensor([2, 3, 2, 1, 3, 2, 3, 3, 3, 1, 3, 1, 2, 1, 2, 3, 1, 2, 2, 3, 2, 2, 2, 1,
        2, 2, 3, 2, 3, 3, 2, 3, 2, 2, 3, 2, 2, 2, 2, 3, 3, 3, 2, 1, 1, 3, 1, 2,
        2, 3, 3, 2, 2, 2, 2, 2, 3, 2, 2, 1, 2, 2, 3, 2, 2, 3, 2, 3, 2, 2, 2, 1,
        3, 3, 3, 3, 2, 1, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 2, 3, 3, 2, 2, 2,
        2, 3, 1, 3, 2, 3, 1, 2, 2, 1, 3, 3, 1, 3, 3, 2, 1, 2, 1, 2, 3, 2, 3, 3,
        3, 2, 2, 3, 2, 3, 2, 3, 2, 2, 2, 3, 3, 3, 2, 3, 2, 3, 2, 1, 2, 3, 2, 2,
        2, 3, 2, 2, 3, 2, 3, 3, 1, 2, 3, 1, 2, 2, 2, 2, 2, 2, 1, 3, 1, 2, 3, 2,
        2, 2, 2, 3, 3, 2, 3, 2, 3, 2, 2, 2, 3, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3,
        2, 3, 2, 2, 2, 2, 2, 3], device='cuda:0')
tensor([2, 2, 3, 2, 3, 2, 2, 1, 3, 1, 3, 2, 3, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 1, 3, 3, 3, 3, 2, 3, 3, 2, 2, 3, 3, 2, 0, 2, 1, 3, 3, 2, 3, 2,
        3, 2, 3, 2, 3, 2, 1, 3, 2, 2, 2, 3, 2, 2, 3, 3, 2, 0, 3, 2, 2, 3, 2, 3,
        2, 3, 2, 2, 0, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2,

tensor([2, 1, 1, 3, 2, 3, 2, 2, 3, 1, 2, 2, 2, 3, 2, 2, 1, 2, 3, 2, 2, 2, 2, 3,
        3, 2, 2, 2, 1, 1, 1, 3, 3, 2, 1, 1, 2, 1, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2,
        2, 2, 3, 1, 2, 1, 3, 2, 2, 2, 2, 2, 1, 3, 1, 2, 3, 2, 3, 2, 2, 3, 2, 3,
        3, 3, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 1, 2, 3, 3, 2, 2,
        3, 3, 3, 2, 1, 2, 3, 2, 2, 3, 2, 2, 3, 1, 1, 1, 2, 3, 2, 3, 3, 3, 1, 3,
        3, 2, 2, 3, 2, 2, 1, 2, 0, 2, 2, 3, 2, 3, 2, 2, 2, 3, 2, 2, 2, 3, 2, 1,
        2, 1, 2, 3, 2, 2, 2, 3, 3, 1, 2, 3, 2, 2, 2, 2, 1, 3, 3, 1, 2, 3, 2, 1,
        1, 2, 2, 3, 1, 2, 2, 1, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2,
        3, 0, 2, 1, 2, 3, 2, 2], device='cuda:0')
tensor([1, 2, 3, 1, 2, 2, 3, 1, 3, 2, 2, 3, 2, 3, 1, 2, 2, 2, 3, 2, 2, 2, 1, 3,
        3, 3, 2, 1, 3, 2, 3, 2, 1, 2, 1, 3, 2, 2, 3, 2, 2, 3, 3, 3, 1, 3, 3, 1,
        3, 3, 1, 1, 2, 1, 2, 1, 2, 3, 1, 2, 2, 2, 3, 2, 3, 1, 3, 2, 2, 2, 2, 3,
        2, 3, 2, 3, 2, 3, 2, 2, 2, 2, 3, 2, 3, 1, 2, 2, 3, 2, 2, 3, 3,

tensor([3, 2, 2, 2, 3, 2, 3, 2, 3, 3, 2, 3, 1, 1, 3, 2, 3, 1, 2, 2, 2, 2, 3, 3,
        0, 3, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 3, 3, 2, 2, 1,
        3, 1, 1, 3, 1, 3, 3, 2, 1, 2, 2, 3, 2, 3, 3, 3, 3, 2, 3, 3, 2, 2, 2, 3,
        1, 3, 3, 2, 1, 2, 3, 2, 3, 3, 2, 2, 2, 2, 1, 1, 3, 2, 1, 2, 2, 2, 3, 2,
        3, 2, 2, 2, 3, 3, 2, 3, 3, 1, 2, 3, 2, 1, 3, 2, 3, 3, 3, 1, 3, 2, 2, 2,
        1, 2, 1, 2, 2, 2, 3, 1, 2, 2, 2, 3, 3, 3, 3, 2, 1, 2, 3, 2, 3, 3, 2, 0,
        2, 3, 2, 2, 2, 3, 1, 2, 3, 2, 3, 3, 2, 2, 2, 2, 2, 1, 2, 2, 1, 3, 1, 1,
        3, 2, 1, 0, 3, 1, 3, 3, 2, 1, 2, 3, 1, 1, 2, 1, 1, 2, 2, 3, 2, 3, 3, 3,
        2, 3, 2, 1, 3, 2, 3, 2], device='cuda:0')
tensor([2, 1, 2, 2, 2, 2, 3, 3, 3, 1, 3, 1, 1, 1, 2, 3, 2, 3, 2, 2, 1, 2, 3, 3,
        2, 2, 1, 2, 3, 2, 3, 3, 3, 3, 3, 1, 3, 3, 1, 1, 1, 2, 1, 3, 3, 1, 2, 3,
        3, 3, 2, 1, 3, 2, 3, 3, 2, 2, 2, 1, 0, 3, 2, 2, 1, 2, 3, 3, 1, 2, 2, 2,
        2, 1, 3, 2, 2, 3, 3, 3, 2, 1, 3, 2, 3, 1, 2, 2, 2, 3, 3, 2, 2,

tensor([2, 1, 3, 3, 3, 2, 2, 2, 2, 3, 2, 1, 3, 2, 3, 1, 2, 2, 2, 3, 2, 2, 3, 2,
        2, 1, 3, 3, 1, 0, 3, 2, 2, 2, 2, 3, 2, 2, 1, 2, 2, 2, 3, 2, 3, 2, 2, 3,
        1, 2, 3, 3, 3, 2, 2, 3, 3, 2, 2, 3, 3, 2, 2, 2, 2, 2, 3, 2, 3, 2, 3, 1,
        3, 2, 2, 3, 2, 2, 3, 2, 1, 2, 3, 2, 1, 1, 3, 3, 3, 3, 1, 2, 2, 3, 2, 1,
        3, 3, 2, 2, 2, 2, 2, 3, 1, 2, 2, 3, 3, 2, 2, 3, 2, 2, 2, 2, 3, 3, 1, 2,
        3, 3, 2, 3, 2, 3, 2, 3, 3, 2, 3, 2, 2, 2, 1, 2, 1, 3, 2, 3, 3, 3, 3, 2,
        2, 2, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 3, 2, 2, 2, 1, 3, 2, 1,
        3, 3, 2, 2, 1, 2, 2, 3, 3, 2, 2, 3, 2, 3, 3, 2, 2, 2, 3, 3, 3, 3, 2, 3,
        3, 2, 2, 1, 3, 1, 0, 2], device='cuda:0')
tensor([3, 2, 2, 2, 1, 3, 3, 2, 2, 3, 2, 1, 3, 2, 3, 3, 3, 1, 2, 2, 2, 1, 2, 2,
        2, 2, 2, 3, 3, 1, 3, 1, 1, 3, 3, 3, 1, 2, 3, 2, 2, 2, 3, 3, 2, 2, 2, 2,
        3, 2, 2, 2, 2, 1, 3, 2, 3, 2, 3, 3, 2, 2, 3, 3, 3, 2, 3, 2, 2, 3, 3, 2,
        2, 2, 3, 2, 1, 2, 2, 3, 3, 3, 3, 3, 2, 2, 2, 2, 3, 3, 1, 2, 3,

tensor([2, 1, 2, 3, 1, 3, 2, 3, 2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 3, 3,
        1, 3, 3, 3, 2, 2, 1, 3, 2, 2, 3, 0, 2, 3, 3, 0, 1, 3, 1, 3, 1, 2, 2, 3,
        2, 2, 3, 2, 3, 2, 0, 2, 2, 3, 2, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 3, 3, 2,
        3, 2, 2, 2, 2, 3, 2, 2, 3, 2, 1, 3, 2, 2, 1, 2, 3, 1, 2, 1, 2, 2, 2, 3,
        3, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 3, 3, 2, 3, 3, 1, 3,
        2, 2, 2, 3, 1, 2, 2, 2, 3, 2, 2, 1, 2, 3, 2, 3, 1, 3, 2, 2, 2, 2, 3, 1,
        1, 3, 2, 2, 2, 2, 1, 3, 2, 1, 3, 2, 3, 3, 3, 3, 3, 1, 2, 2, 2, 3, 2, 3,
        2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 2, 3, 2, 3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 3], device='cuda:0')
tensor([2, 3, 2, 3, 3, 2, 2, 3, 3, 2, 2, 3, 3, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 0,
        3, 3, 3, 1, 3, 2, 3, 3, 3, 2, 2, 2, 3, 1, 3, 1, 2, 1, 2, 2, 2, 2, 1, 2,
        2, 1, 3, 3, 1, 2, 2, 2, 1, 3, 2, 3, 2, 3, 2, 3, 3, 2, 2, 3, 1, 1, 1, 2,
        3, 2, 0, 2, 2, 2, 3, 3, 3, 3, 2, 2, 3, 2, 1, 2, 1, 1, 2, 1, 2,

...

tensor([3, 3, 1, 2, 3, 2, 2, 3, 1, 3, 3, 2, 2, 2, 3, 2, 3, 3, 3, 1, 2, 2, 2, 2,
        2, 2, 2, 1, 2, 1, 3, 2, 2, 2, 2, 2, 1, 2, 2, 3, 1, 3, 3, 3, 2, 2, 2, 2,
        3, 2, 3, 2, 3, 2, 2, 3, 3, 1, 3, 3, 3, 1, 3, 3, 2, 3, 1, 2, 3, 2, 3, 2,
        2, 2, 1, 2, 2, 2, 3, 2, 2, 1, 2, 2, 2, 1, 2, 3, 2, 3, 3, 3, 2, 2, 3, 1,
        3, 0, 2, 3, 2, 2, 3, 2, 1, 2, 1, 1, 2, 3, 2, 3, 3, 3, 1, 2, 1, 2, 1, 3,
        2, 2, 3, 3, 2, 2, 1, 3, 1, 2, 1, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2,
        1, 2, 3, 2, 2, 2, 2, 3, 3, 2, 1, 2, 3, 1, 3, 2, 2, 3, 2, 2, 1, 2, 3, 3,
        2, 2, 2, 1, 3, 3, 3, 3, 2, 2, 3, 1, 1, 2, 1, 3, 3, 3, 3, 3, 1, 3, 1, 3,
        2, 3, 3, 2, 3, 3, 3, 2], device='cuda:0')
tensor([1, 2, 2, 2, 3, 3, 1, 2, 3, 3, 3, 2, 2, 2, 3, 3, 2, 2, 2, 1, 3, 2, 3, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 1, 3, 2, 3, 3, 2, 2, 1, 1, 3,
        2, 2, 2, 3, 3, 2, 3, 2, 2, 2, 1, 2, 3, 2, 2, 3, 2, 3, 1, 2, 3, 2, 2, 2,
        3, 1, 3, 3, 1, 2, 3, 3, 3, 2, 2, 3, 3, 2, 2, 2, 2, 3, 2, 2, 3,

tensor([2, 1, 3, 3, 1, 2, 2, 2, 3, 1, 2, 3, 2, 2, 3, 3, 2, 2, 3, 2, 1, 2, 2, 2,
        1, 3, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 3, 3, 3, 1, 1,
        3, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3,
        2, 2, 2, 1, 3, 2, 1, 2, 3, 1, 2, 3, 3, 3, 3, 3, 3, 2, 3, 1, 2, 3, 3, 2,
        1, 2, 2, 1, 3, 3, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 3, 2, 1, 0, 2, 2, 2,
        3, 2, 2, 2, 2, 3, 1, 3, 3, 2, 2, 3, 2, 3, 3, 3, 2, 2, 1, 2, 2, 2, 2, 2,
        1, 3, 2, 0, 3, 2, 1, 2, 2, 3, 1, 1, 3, 2, 3, 2, 1, 3, 2, 3, 3, 3, 3, 2,
        1, 3, 2, 3, 2, 2, 3, 3, 1, 2, 2, 3, 2, 3, 2, 2, 1, 3, 0, 3, 2, 3, 3, 2,
        3, 1, 2, 2, 3, 3, 2, 2], device='cuda:0')
tensor([2, 3, 2, 2, 1, 3, 3, 2, 3, 1, 2, 3, 2, 1, 0, 3, 3, 3, 3, 3, 1, 2, 3, 2,
        2, 3, 1, 3, 2, 3, 1, 3, 2, 3, 3, 3, 2, 2, 1, 1, 3, 3, 3, 3, 2, 3, 2, 2,
        2, 1, 2, 1, 1, 3, 3, 3, 2, 2, 3, 2, 1, 2, 2, 2, 3, 3, 2, 2, 2, 3, 2, 2,
        2, 2, 2, 2, 3, 2, 2, 1, 2, 2, 3, 3, 2, 3, 2, 1, 3, 3, 2, 2, 2,

tensor([3, 2, 3, 0, 2, 2, 2, 3, 3, 3, 1, 3, 2, 3, 2, 2, 2, 3, 2, 3, 2, 2, 3, 2,
        3, 1, 2, 2, 1, 3, 3, 2, 3, 2, 2, 0, 3, 2, 2, 3, 3, 2, 3, 2, 1, 3, 3, 3,
        1, 2, 2, 3, 3, 3, 2, 2, 2, 2, 2, 1, 3, 3, 3, 2, 2, 1, 2, 3, 2, 2, 2, 2,
        2, 1, 2, 3, 2, 3, 1, 2, 3, 2, 1, 1, 2, 2, 3, 3, 3, 3, 3, 3, 2, 1, 3, 2,
        2, 2, 2, 2, 1, 2, 3, 1, 2, 1, 3, 3, 2, 1, 3, 1, 2, 2, 2, 3, 3, 2, 2, 3,
        2, 2, 1, 2, 3, 1, 1, 2, 3, 3, 3, 2, 3, 3, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2,
        2, 2, 1, 2, 2, 2, 3, 2, 2, 2, 2, 1, 3, 3, 3, 2, 2, 3, 3, 2, 1, 2, 2, 3,
        3, 2, 1, 2, 1, 2, 2, 3, 2, 3, 3, 2, 3, 3, 2, 2, 2, 1, 2, 3, 1, 2, 2, 2,
        2, 3, 3, 2, 1, 2, 2, 2], device='cuda:0')
tensor([1, 2, 2, 1, 2, 1, 3, 2, 3, 2, 2, 3, 1, 3, 2, 2, 2, 2, 1, 3, 3, 2, 3, 1,
        1, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 3, 2, 3, 2, 2, 3, 3, 2,
        2, 2, 3, 2, 2, 1, 2, 2, 2, 2, 3, 3, 2, 3, 3, 2, 3, 2, 2, 3, 3, 1, 2, 3,
        3, 3, 1, 2, 2, 3, 2, 2, 3, 1, 2, 1, 3, 2, 2, 2, 2, 2, 2, 3, 2,

tensor([2, 3, 3, 1, 1, 2, 3, 3, 2, 3, 3, 3, 2, 1, 3, 2, 1, 1, 2, 1, 2, 2, 3, 2,
        2, 3, 2, 3, 2, 2, 2, 3, 2, 3, 1, 2, 3, 3, 2, 2, 2, 3, 2, 3, 2, 2, 3, 2,
        3, 2, 3, 2, 3, 1, 2, 2, 2, 2, 2, 3, 1, 1, 3, 1, 3, 3, 2, 2, 2, 1, 1, 2,
        3, 2, 3, 2, 3, 3, 3, 2, 3, 1, 1, 3, 1, 3, 2, 3, 2, 2, 2, 1, 2, 2, 2, 3,
        2, 3, 3, 3, 2, 1, 3, 3, 3, 2, 3, 1, 3, 3, 1, 2, 2, 2, 2, 3, 2, 2, 2, 2,
        2, 2, 2, 3, 1, 1, 1, 1, 2, 3, 2, 3, 2, 2, 2, 2, 3, 1, 2, 3, 1, 2, 3, 3,
        3, 1, 2, 3, 1, 2, 1, 3, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 2, 1, 2, 2, 1,
        2, 3, 2, 2, 3, 1, 2, 3, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2,
        2, 3, 1, 3, 2, 2, 2, 3], device='cuda:0')
tensor([2, 2, 3, 3, 2, 3, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 3, 3, 2, 1, 2, 2, 3, 3,
        3, 2, 2, 2, 1, 3, 1, 3, 3, 2, 2, 3, 3, 2, 3, 2, 1, 3, 2, 3, 3, 2, 3, 3,
        3, 2, 2, 2, 2, 2, 0, 3, 2, 2, 1, 2, 2, 3, 3, 2, 1, 3, 2, 3, 3, 3, 3, 3,
        3, 2, 3, 3, 3, 1, 2, 2, 3, 2, 2, 3, 3, 2, 1, 2, 2, 2, 2, 3, 2,

tensor([2, 2, 3, 1, 3, 2, 2, 3, 3, 3, 3, 2, 2, 1, 1, 2, 3, 3, 2, 3, 2, 2, 2, 2,
        3, 2, 2, 2, 2, 1, 2, 2, 3, 2, 2, 2, 3, 2, 2, 1, 3, 0, 2, 2, 2, 3, 2, 2,
        2, 3, 2, 2, 2, 3, 2, 3, 3, 3, 2, 2, 2, 3, 1, 2, 3, 2, 2, 3, 3, 2, 2, 3,
        2, 2, 2, 2, 3, 3, 2, 2, 2, 1, 3, 2, 2, 1, 3, 3, 3, 3, 3, 0, 1, 3, 2, 2,
        1, 1, 2, 3, 2, 3, 0, 2, 2, 3, 2, 2, 3, 3, 3, 3, 1, 2, 1, 2, 2, 1, 3, 2,
        3, 2, 1, 3, 2, 2, 2, 2, 1, 2, 3, 2, 2, 1, 3, 2, 3, 2, 3, 2, 3, 3, 1, 2,
        2, 3, 1, 2, 3, 3, 1, 2, 3, 2, 1, 1, 1, 2, 0, 3, 1, 3, 2, 2, 1, 2, 2, 3,
        1, 2, 3, 2, 2, 2, 2, 3, 2, 1, 2, 3, 2, 3, 2, 1, 1, 2, 2, 2, 3, 2, 2, 2,
        3, 2, 3, 3, 1, 3, 3, 3], device='cuda:0')
tensor([1, 2, 3, 3, 3, 2, 3, 2, 3, 2, 3, 3, 2, 2, 2, 3, 2, 1, 3, 2, 0, 2, 2, 3,
        3, 3, 2, 2, 2, 3, 3, 1, 3, 1, 2, 3, 2, 2, 0, 3, 3, 3, 3, 2, 3, 2, 2, 2,
        2, 1, 2, 2, 2, 2, 3, 3, 3, 2, 3, 2, 2, 3, 2, 2, 2, 2, 1, 1, 1, 3, 2, 3,
        2, 3, 3, 3, 2, 3, 3, 2, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 2, 2,

In [11]:
import random

In [31]:
model_ft.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [32]:
def predict_category(image):
    # receives (M,N,3) image and returns predicted category
    # print(np.shape(image),end='\t')
    data=torch.tensor(io.imread(image).transpose((2, 0, 1))).view(1,3,60,60)
    outputs = model_ft(data.float().to(device,dtype=torch.float32))
    # print(outputs)
    _, preds = torch.max(outputs, 1)
    return preds.cpu().numpy()[0]

In [33]:
def draw_layer(draw=50,folder='layers',home='problem2',resize=0.02,crop_x=60,crop_y=60,cooling_alpha=0.001):
    # create directory
    if not os.path.exists('./%s'%(folder)): os.mkdir('./%s'%(folder))
    if not os.path.exists('./%s/draw'%(folder)): os.mkdir('./%s/draw'%(folder))

    # read label
    layer='L%04d'%(draw)
#     mpm_dir='./%s/MPM/Layer%03dto%03d/%s/'%(home,(draw-1)//50*50+1,(draw-1)//50*50+50,layer)
#     pixel=[]
#     for temp in range(len(os.listdir(mpm_dir))):
#         name='%sframe_%04d.bmp'%(mpm_dir,temp+1)
#         pixel.append(np.sum(io.imread(name)>150))
#     threshold=np.array([10,np.quantile(pixel,.25),np.quantile(pixel,.75)])
#     category=[np.sum(temp>threshold) for temp in pixel]

    # read cmd
    cmd=pd.read_csv('./%s/Command_Part1/XYPT_Part01_%s.csv'%(home,layer),header=None)
    left=cmd[0].min()
    right=cmd[0].max()
    down=cmd[1].min()
    up=cmd[1].max()
    mag=1000
    width=int((right-left)*mag*resize)+1
    height=int((up-down)*mag*resize)+1
    power=np.zeros([width,height])
    speed=np.zeros([width,height])
    temporal=np.zeros([width,height])

    current_order=0
    locations=[]
    loc_x=[]
    loc_y=[]
    first_row=cmd[0:1]
    history_x=[first_row[0][0]]*3
    history_y=[first_row[1][0]]*3
    speed_max=-10000
    speed_min=10000
    for line in cmd.iterrows():
        raw_x=line[1][0]
        raw_y=line[1][1]
        history_x.pop()
        history_x.insert(0,raw_x)
        history_y.pop()
        history_y.insert(0,raw_y)
        # calculate speed
        last_speed=((history_x[2]-history_x[0])*(history_x[2]-history_x[0])+(history_y[2]-history_y[0])*(history_y[2]-history_y[0]))**0.5/2e-5
        if current_order>0:
            speed[current_x,current_y]=last_speed
            speed_max=max(last_speed,speed_max)
            speed_min=min(last_speed,speed_min)
        current_order+=1
        current_x=int((raw_x-left)*mag*resize)
        current_y=int((raw_y-down)*mag*resize)
        power[current_x,current_y]=line[1][2]
        temporal[current_x,current_y]=current_order
        if line[1][3]==2:
            locations.append([current_x,current_y])
            loc_x.append(raw_x)
            loc_y.append(raw_y)
    power_min=np.min(power)
    power_max=np.max(power)
    
    # generate photos
    photo_id=0
    for temp in locations:
        if photo_id%100==0: print(photo_id,end='\t')
        center_x,center_y=temp
        power_crop=np.zeros([crop_x,crop_y])
        speed_crop=np.zeros([crop_x,crop_y])
        temporal_crop=np.zeros([crop_x,crop_y])
        for x in range(center_x-int(crop_x*0.5),center_x+int(crop_x*0.5)):
            for y in range(center_y-int(crop_y*0.5),center_y+int(crop_y*0.5)):
                # consider locations in the window with time stamp earlier than center
                if x<0 or x>=width: continue
                if y<0 or y>=height: continue
                if temporal[x,y]==0 or temporal[x,y]>temporal[center_x,center_y]: continue
                # power crop
                original_power=(power[x,y]-power_min)/(power_max-power_min)
                power_crop[x+int(crop_x*0.5)-center_x,y+int(crop_y*0.5)-center_y]=int(original_power*255)
                # speed crop
                original_speed=(speed[x,y]-speed_min)/(speed_max-speed_min)
                speed_crop[x+int(crop_x*0.5)-center_x,y+int(crop_y*0.5)-center_y]=int(original_speed*255)
                # temporal crop
                elapsed_time=temporal[center_x,center_y]-temporal[x,y]
                temporal_crop[x+int(crop_x*0.5)-center_x,y+int(crop_y*0.5)-center_y]=int(np.exp(-elapsed_time*cooling_alpha)*255)
        rgb_image=np.stack((power_crop.astype(np.uint8),speed_crop.astype(np.uint8),temporal_crop.astype(np.uint8)),axis=-1)
        io.imsave('./%s/draw/%d.png'%(folder,photo_id),rgb_image,check_contrast=False)
        photo_id+=1

    pixel_results=[]
    category_results=[]
    for photo in range(photo_id):
        if photo%100==0:print(photo,end='\t')
        img=predict_category('./%s/draw/%d.png'%(folder,photo))
        # pixel_results.append(img)
        # predict category
        category_results.append(img)
    # use quantiles to predict category
    # threshold_results=np.array([10,np.quantile(pixel_results,.25),np.quantile(pixel_results,.75)])
    # category_results=[np.sum(temp>threshold_results) for temp in pixel_results]

    pd.DataFrame({'x':loc_x,'y':loc_y,
                #   'pixel_label':pixel,
#                   'category_label':category,
                #   'pixel_predict':pixel_results,
                  'category_predict':category_results}).to_csv('./%s/heatmap_%d.csv'%(folder,draw),index=0)

In [34]:
draw_layer(draw=100)

0	100	200	300	400	500	600	700	800	900	1000	1100	1200	1300	1400	1500	1600	1700	1800	1900	2000	2100	2200	2300	2400	2500	2600	2700	2800	2900	3000	3100	3200	3300	3400	3500	3600	0	100	200	300	400	500	600	700	800	900	1000	1100	1200	1300	1400	1500	1600	1700	1800	1900	2000	2100	2200	2300	2400	2500	2600	2700	2800	2900	3000	3100	3200	3300	3400	3500	3600	